In [1]:
!pip install xgboost

In [3]:
import xgboost as xgb
import numpy as np
from collections import OrderedDict
import gc
from glob import glob
import os
import pandas as pd
from copy import copy
from time import time
from sklearn.metrics import roc_auc_score,confusion_matrix,accuracy_score,classification_report,roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from timeit import default_timer
import matplotlib.pyplot as plt
import pickle

In [4]:
np.random.seed(123)

In [6]:
data=pd.read_csv('/content/Test_Dataset_FVPL.csv')

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2505 entries, 0 to 2504
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   timeindex               2505 non-null   int64  
 1   currentBack             2505 non-null   float64
 2   motorTempBack           2505 non-null   int64  
 3   positionBack            2505 non-null   float64
 4   refPositionBack         2505 non-null   float64
 5   refVelocityBack         2505 non-null   float64
 6   trackingDeviationBack   2505 non-null   float64
 7   velocityBack            2505 non-null   float64
 8   currentFront            2505 non-null   float64
 9   motorTempFront          2505 non-null   int64  
 10  positionFront           2505 non-null   float64
 11  refPositionFront        2505 non-null   float64
 12  refVelocityFront        2505 non-null   float64
 13  trackingDeviationFront  2505 non-null   float64
 14  velocityFront           2505 non-null   

In [11]:
data.isnull().sum()

timeindex                 0
currentBack               0
motorTempBack             0
positionBack              0
refPositionBack           0
refVelocityBack           0
trackingDeviationBack     0
velocityBack              0
currentFront              0
motorTempFront            0
positionFront             0
refPositionFront          0
refVelocityFront          0
trackingDeviationFront    0
velocityFront             0
dtype: int64

In [7]:
data

,timeindex,currentBack,motorTempBack,positionBack,refPositionBack,refVelocityBack,trackingDeviationBack,velocityBack,currentFront,motorTempFront,positionFront,refPositionFront,refVelocityFront,trackingDeviationFront,velocityFront
0,1,-37.5,43,6.999564,5.000000,8.0,10.000478,4.979237,0.0,42,401.000987,399.000000,8.0,4.999003,7.979237
1,2,-11.5,47,580.499086,581.499997,6.0,9.000914,6.000000,25.0,44,973.498993,972.499997,6.0,7.000997,8.000000
2,3,68.0,44,576.500041,579.499998,10.0,9.999917,6.031144,142.0,49,971.499450,970.499998,7.0,8.000498,9.000000
3,4,136.0,44,579.499709,580.499998,5.0,8.000291,6.979237,137.0,48,970.499616,969.499998,7.0,6.000374,9.000000
4,5,66.0,47,579.499709,580.499998,5.0,5.000291,7.000000,123.0,44,971.499450,971.499998,5.0,9.000540,8.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2500,2501,240.0,44,579.500436,578.499997,8.0,4.986234,8.020763,354.0,46,970.511949,969.499997,5.0,7.986712,8.041526
2501,2502,-14.0,41,4.999626,5.000000,9.0,9.000374,6.000000,41.0,45,397.998578,401.000000,7.0,7.001412,5.000000
2502,2503,327.0,46,577.500540,581.499998,10.0,6.988954,8.228391,328.5,42,969.141374,968.338455,7.0,8.983224,12.259768
2503,2504,-52.0,43,578.502201,579.499997,10.0,4.997799,9.937711,1010.0,42,968.503395,972.499997,9.0,4.996595,6.000000


In [8]:
data1=pd.read_csv('/content/Train_FVPL.csv')

In [ ]:
data1.isnull().sum()

timeindex                 0
flag                      0
currentBack               0
motorTempBack             0
positionBack              0
refPositionBack           0
refVelocityBack           0
trackingDeviationBack     0
velocityBack              0
currentFront              0
motorTempFront            0
positionFront             0
refPositionFront          0
refVelocityFront          0
trackingDeviationFront    0
velocityFront             0
dtype: int64

In [ ]:
data1.isnull().sum()

timeindex                 0
flag                      0
currentBack               0
motorTempBack             0
positionBack              0
refPositionBack           0
refVelocityBack           0
trackingDeviationBack     0
velocityBack              0
currentFront              0
motorTempFront            0
positionFront             0
refPositionFront          0
refVelocityFront          0
trackingDeviationFront    0
velocityFront             0
dtype: int64

In [13]:
data1.isnull().sum()

timeindex                 0
flag                      0
currentBack               0
motorTempBack             0
positionBack              0
refPositionBack           0
refVelocityBack           0
trackingDeviationBack     0
velocityBack              0
currentFront              0
motorTempFront            0
positionFront             0
refPositionFront          0
refVelocityFront          0
trackingDeviationFront    0
velocityFront             0
dtype: int64

In [9]:
data1

,timeindex,flag,currentBack,motorTempBack,positionBack,refPositionBack,refVelocityBack,trackingDeviationBack,velocityBack,currentFront,motorTempFront,positionFront,refPositionFront,refVelocityFront,trackingDeviationFront,velocityFront
0,1,1,30.0,46,576.499377,581.499997,7.000000,7.000623,10.000000,16.0,46,970.499907,971.499997,10.000000,8.000125,5.000000
1,2,1,101.0,42,576.499211,576.499999,9.000000,8.000747,7.000000,66.0,42,969.499450,969.499999,5.000000,10.000540,10.000000
2,3,1,815.0,41,355.576792,363.368778,584.045836,4.994664,587.020659,726.0,38,741.523412,742.361361,590.807348,4.987812,593.415386
3,4,1,219.0,40,577.499585,578.499998,7.000000,6.000332,8.041526,207.0,39,972.499575,971.499998,5.000000,7.000457,6.031144
4,5,1,76.0,40,4.998796,9.000000,6.000000,7.001246,6.000000,-13.0,43,398.999076,398.000000,6.000000,5.000914,8.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11222,11223,0,-34.0,33,576.499959,578.499997,9.000000,8.000000,5.979237,139.0,33,969.499450,970.499997,8.000000,5.000581,9.000000
11223,11224,0,611.0,39,385.941754,389.770293,588.043809,5.998235,587.778503,718.0,37,761.058641,762.869655,595.118577,5.991404,594.754822
11224,11225,0,-28.0,38,577.500291,581.499997,10.000000,5.999709,8.979237,-18.0,38,971.500073,968.499997,5.000000,10.000000,7.000000
11225,11226,0,-21.0,37,577.499959,576.499997,5.000000,10.000000,6.020763,20.0,36,969.499741,969.499997,7.000000,6.000249,5.000000


In [10]:
from sklearn.model_selection import KFold

In [17]:
y=data1['flag']
x=data1.drop('flag',axis=1)

In [18]:
from sklearn.tree import DecisionTreeClassifier

In [19]:
x.shape, y.shape

((11227, 15), (11227,))

In [20]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,
                                                 test_size=0.30,
                                                 random_state=34)

In [21]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((7858, 15), (3369, 15), (7858,), (3369,))

In [22]:
model = DecisionTreeClassifier()

In [23]:
model

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [25]:
model.fit(x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [26]:
np.round(model.score(x_train,y_train),2)

1.0

In [27]:
np.round(model.score(x_test,y_test),2)

0.86

In [28]:
from sklearn.model_selection import cross_val_score

In [29]:
model3 = DecisionTreeClassifier(max_depth=5)

In [30]:
scores = cross_val_score(estimator= model,     # Model to test
                X= x_train,  
                y = y_train,      # Target variable
                scoring = "accuracy",               # Scoring metric    
                cv=10)                              # Cross validation folds

print("Accuracy per fold: ")
print(scores)
print("Average accuracy: ", scores.mean())
print('Deviation: ', scores.std())

Accuracy per fold: 
[0.86641221 0.8524173  0.86768448 0.86259542 0.85496183 0.83969466
 0.82824427 0.86895674 0.88025478 0.85605096]
Average accuracy:  0.8577272653603669
Deviation:  0.014369893363238549


In [31]:
from sklearn.linear_model import LogisticRegression

In [32]:
model1=LogisticRegression()

In [33]:
model1.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [34]:
y_pred=model1.predict(x_test)

In [35]:
model1.score(x_test,y_test)

0.7598693974473137

In [36]:
model1.score(x_train,y_train)

0.7604988546703996

In [37]:
from sklearn.model_selection import GridSearchCV


In [38]:
clf=DecisionTreeClassifier()

In [39]:
param_grid = {"criterion": ["gini","entropy"],
             "max_depth": [2,3,5,7,10],
              
              
              "min_samples_split":[5,10,15,20]
                         
             }

In [40]:
from time import time
from operator import itemgetter

In [41]:
def GridSearch_BestParam(x, y, clf, param_grid,cv=10):
    grid_search = GridSearchCV(clf,
                              param_grid=param_grid,
                              cv=cv)
    start= time()
    grid_search.fit(x,y)
    top_params=grid_search.best_params_
    return top_params

In [42]:
top_param=GridSearch_BestParam(x_train,y_train,clf,param_grid,cv=10)

In [43]:
print(top_param)

{'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 15}


In [44]:
best_dt_model=DecisionTreeClassifier(criterion= 'entropy', max_depth= 10,  min_samples_split= 15)

In [45]:
best_dt_model.fit(x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=15,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [46]:
best_dt_model.score(x_train,y_train)

0.9001018070755917

In [47]:
best_dt_model.score(x_test,y_test)

0.8610863757791629

In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [49]:
clf2=RandomForestClassifier()

In [50]:
rfs=RandomForestClassifier(random_state=123)

In [51]:
param_grid = { 
    
                  "min_samples_split":[5,10,15,20]
,
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [52]:
def GridSearch_BestParam(x, y, clf2, param_grid,cv=10):
    grid_search = GridSearchCV(clf,
                              param_grid=param_grid,
                              cv=cv)
    start= time()
    grid_search.fit(x,y)
    top_params=grid_search.best_params_
    return top_params

In [53]:
CV_rfc = GridSearchCV(estimator=rfs, param_grid=param_grid, cv= 10)

In [54]:
top_param=GridSearch_BestParam(x_train,y_train,clf2,param_grid,cv=10)

In [55]:
print(top_param)

{'criterion': 'entropy', 'max_depth': 8, 'min_samples_split': 10}


In [56]:
model = RandomForestClassifier(criterion= 'entropy',
 max_depth= 8,
 min_samples_split=10,
 )

In [57]:
model.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=8, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [58]:
model.score(x_train,y_train)

0.8605243064392976

In [59]:
model.score(x_test,y_test)

0.8379341050756901

In [61]:
model = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=8, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [62]:
model.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=8, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [63]:
model.score(x_train,y_train)

0.8513616696360397

In [64]:
model.score(x_test,y_test)

0.8349658652419115